In [1]:
#default_exp github_crawling

In [7]:
#export
import pandas as pd
import requests
import json
from operator import itemgetter
import tqdm

In [11]:
#export


def get_python_files(owner, repo_name):
    url_template = 'https://api.github.com/repos/{}/{}/git/trees/master?recursive=1'
    files = json.loads(requests.get(url_template.format(owner, repo_name)).text)['tree']

    for maybe_file in files:
        if maybe_file['type'] == 'blob' and maybe_file['path'][-3:] == '.py':
            path = maybe_file['path']
            raw_file_url_template = 'https://raw.githubusercontent.com/{}/{}/master/{}'
            raw_file_url = raw_file_url_template.format(owner, repo_name, path)
            yield owner, repo_name, path, requests.get(raw_file_url).text, maybe_file['sha'] 

In [12]:
example_owner = 'lambdaofgod'
example_repo = 'mlutil'

In [13]:
#export


def get_python_files_df(owner, repo_name):
    file_tuples = list(get_python_files(owner, repo_name))
    if len(file_tuples) > 0:
        df = pd.DataFrame.from_records(file_tuples)
        df.columns = ['owner', 'repo_name', 'file_path', 'content', 'sha']
        return df
    else:
        return pd.DataFrame({})

In [14]:
example_files_df = get_python_files_df(example_owner, example_repo)

In [17]:
example_files_df.head()['content'].iloc[1]

'import gzip\nfrom typing import Dict, Callable\nimport numpy as np\n\n\nclass CompressedKeyedVectors(object):\n\n    def __init__(self, vocab_path: str, embedding_path: str, to_lowercase: bool=True):\n        """\n        Class from sdadas polish-nlp-resources\n        https://github.com/sdadas/polish-nlp-resources\n        I need to get it somewhere from where I can import it easily for using with custom BentoML model\n        """\n        self.vocab_path: str = vocab_path\n        self.embedding_path: str = embedding_path\n        self.to_lower: bool = to_lowercase\n        self.vocab: Dict[str, int] = self.__load_vocab(vocab_path)\n        embedding = np.load(embedding_path)\n        self.codes: np.ndarray = embedding[embedding.files[0]]\n        self.codebook: np.ndarray = embedding[embedding.files[1]]\n        self.m = self.codes.shape[1]\n        self.k = int(self.codebook.shape[0] / self.m)\n        self.dim: int = self.codebook.shape[1]\n\n    def __load_vocab(self, vocab_path